In [1]:
import sys
import pandas as pd
sys.path.append('..')
from data_feature_extraction.CoT_Dissa import extract_data

In [2]:
# df = extract_data('../data/GC=F_com_disagg.csv', '../data/GC=F_com_disagg_finalextracted.csv')
df = pd.read_csv('../data/GC=F_com_disagg_finalextracted.csv')

In [3]:
string_cols = df.select_dtypes(include=['object']).columns.tolist()
df.set_index(['Date'])
# Drop the string columns from the DataFrame
df_ = df.drop(columns=string_cols, inplace=False)
df_['Date'] = pd.to_datetime(df['Date'])
df_.set_index(['Date'],drop=True, inplace=True)
df_.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Open_Interest_All', 'Swap_Positions_Long_All',
       'Swap__Positions_Short_All', 'M_Money_Positions_Short_All',
       'Other_Rept_Positions_Long_All', 'NonRept_Positions_Short_All',
       'Pct_of_OI_Prod_Merc_Short_All', 'Pct_of_OI_Swap_Short_All',
       'Pct_of_OI_Other_Rept_Short_All', 'Pct_of_OI_Tot_Rept_Short_All',
       'Traders_Tot_All', 'Traders_M_Money_Long_All',
       'Conc_Gross_LE_4_TDR_Long_All', 'Conc_Net_LE_4_TDR_Long_All',
       'Conc_Net_LE_4_TDR_Short_All', 'volume_adi', 'volume_obv',
       'volume_vwap', 'volume_nvi', 'volatility_atr'],
      dtype='object')

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, df, sequence_length):
        self.df = df
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.df) - self.sequence_length

    def __getitem__(self, index):
        sequence_data = self.df.iloc[index: index + self.sequence_length]
        sequence_target = self.df.iloc[index + self.sequence_length]
        
        # You will have to adjust the following lines based on the 
        # specific structure of your dataframe and the type of model you're building
        x = torch.tensor(sequence_data.values).float()
        y = torch.tensor(sequence_target.values).float()

        return x, y
    

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

c:\Users\lqs\miniconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class StockPriceTransformer(nn.Module):
    def __init__(self, feature_size, d_model, nhead, num_layers, dropout):
        super(StockPriceTransformer, self).__init__()
        self.input_linear = nn.Linear(feature_size, d_model)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=d_model*4, dropout=dropout), 
            num_layers)
        self.output_linear = nn.Linear(d_model, 1)  # Predicting one value (next day's close price)

    def forward(self, src):
        src = self.input_linear(src)
        src = src.transpose(0, 1)  # Transformer expects seq_len, batch, features
        output = self.transformer_encoder(src)
        output = self.output_linear(output[-1])  # We only care about the last output for sequence-to-one.
        return output

In [6]:
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    epoch_loss = 0
    for src, tgt in dataloader:
        src, tgt = src.to(device), tgt.to(device)
        optimizer.zero_grad()
        output = model(src)
        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(dataloader)

def test(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)
            output = model(src)
            loss = criterion(output, tgt)
            epoch_loss += loss.item()
    return epoch_loss / len(dataloader)

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Defining the model hyperparameters
d_model = 512  # The number of expected features in the input, it can vary based on your dataset
nhead = 8  # The number of heads in the multiheadattention models, usually d_model is divisible by nhead
num_layers = 3  # The number of sub-encoder-layers in the transformer encoder
dropout = 0.1  # The dropout value
feature_size = 26
num_epochs = 20

# dataloader configurations
sequence_length = 50  # Define your sequence length
batch_size = 32  # Define your batch size
# Creating a dataset
sequence_length = 50  # Define your sequence length
dataset = TimeSeriesDataset(df_, sequence_length)

# Splitting dataset into training set and testing set
train_size = int(0.8 * len(dataset))  # 80% of the dataset
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Creating dataloaders
batch_size = 32  # Define your batch size

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)




model = StockPriceTransformer(feature_size, d_model, nhead, num_layers, dropout).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    valid_loss = test(model, test_loader, criterion, device)
    print(f"Epoch: {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}")

c:\Users\lqs\miniconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([32, 26])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\lqs\miniconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([31, 26])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1, Train Loss: 81724528298.6667, Validation Loss: 82903004501.3333
Epoch: 2, Train Loss: 81721749845.3333, Validation Loss: 82899981653.3333
Epoch: 3, Train Loss: 81718502741.3333, Validation Loss: 82896373077.3333
Epoch: 4, Train Loss: 81714703360.0000, Validation Loss: 82892201984.0000
Epoch: 5, Train Loss: 81710363648.0000, Validation Loss: 82887494314.6667
Epoch: 6, Train Loss: 81705500672.0000, Validation Loss: 82882244608.0000
Epoch: 7, Train Loss: 81700109312.0000, Validation Loss: 82876465152.0000
Epoch: 8, Train Loss: 81694192298.6667, Validation Loss: 82870153216.0000
Epoch: 9, Train Loss: 81687748266.6667, Validation Loss: 82863301973.3333
Epoch: 10, Train Loss: 81680766293.3333, Validation Loss: 82855897770.6667
Epoch: 11, Train Loss: 81673242965.3333, Validation Loss: 82847948800.0000
Epoch: 12, Train Loss: 81665184426.6667, Validation Loss: 82839474176.0000
Epoch: 13, Train Loss: 81656579072.0000, Validation Loss: 82830401536.0000
Epoch: 14, Train Loss: 81647408810